<a href="https://colab.research.google.com/github/kaushikpichumani/load_query_neo4j_llm/blob/main/connecting_to_neo4j.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!pip install --upgrade --quiet langchain langchain-community langchain-groq neo4j

In [12]:
# Graphdb configuration

# Neo4j credentials
# you can get this upon creating an instence in neo4j

NEO4J_URI="neo4j+s://4689780a.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="dLBGuDmNE6KgPym5gM1uiFxP-JK2iSC4wArdWwfb4W0"

In [13]:
# Set the credentials as environment variable

import os
os.environ["NEO4J_URI"] = NEO4J_URI
os.environ["NEO4J_USERNAME"] = NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"] = NEO4J_PASSWORD

In [14]:
# have an object created with Neo4j Database with provioded credentials
from langchain_community.graphs import Neo4jGraph
graph = Neo4jGraph(
    url=os.environ.get("NEO4J_URI"),
    username=os.environ.get("NEO4J_USERNAME"),
    password=os.environ.get("NEO4J_PASSWORD"),
)


In [15]:

graph

In [16]:
# Create GROQ api key. I am using free model throughout the exercise. Feel free to use paid credentials.
# We can also explore other API like openai etc
# GROQ will be free until certain number of HIT's
groq_api_key = "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"

In [17]:
# initiate the model
from langchain_groq import ChatGroq
llm = ChatGroq(
    groq_api_key=groq_api_key,model="Gemma2-9b-It",
    verbose=True,
)

In [18]:
llm

ChatGroq(verbose=True, client=<groq.resources.chat.completions.Completions object at 0x7ef45511fc70>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7ef4548e8820>, model_name='Gemma2-9b-It', groq_api_key=SecretStr('**********'))

In [44]:
# grabbed this free text from wiki about Elon Musk
# we will try to convert this data and load it into graph db

from langchain_core.documents import Document
text = """Elon Reeve Musk FRS (/ˈiːlɒn/; born June 28, 1971) is a businessman and investor known for
his key roles in space company SpaceX and automotive company Tesla, Inc. Other involvements include ownership of X Corp.,
the company that operates the social media platform X (formerly known as Twitter), and his role in the founding of The Boring Company, xAI,
Neuralink and OpenAI. He is one of the wealthiest people in the world; as of July 2024, Forbes estimates his net worth to be US$221 billion.[4]

Musk was born in Pretoria to model Maye and businessman and engineer Errol Musk, and briefly attended the
University of Pretoria before immigrating to Canada at age 18, acquiring citizenship through his Canadian-born mother. Two years later,
he matriculated at Queen's University at Kingston in Canada. Musk later transferred to the University of Pennsylvania and received bachelor's
degrees in economics and physics. He moved to California in 1995 to attend Stanford University, but dropped out after two days and, with his brother
Kimbal,
co-founded online city guide software company Zip2"""

In [46]:
# converts the document into llm consumable format
documents = [Document(page_content=text)]

In [45]:
documents

[Document(metadata={'id': 'd56a2b900e7aa5825f468827e8186d77'}, page_content="Elon Reeve Musk FRS (/ˈiːlɒn/; born June 28, 1971) is a businessman and investor known for \nhis key roles in space company SpaceX and automotive company Tesla, Inc. Other involvements include ownership of X Corp., \nthe company that operates the social media platform X (formerly known as Twitter), and his role in the founding of The Boring Company, xAI, \nNeuralink and OpenAI. He is one of the wealthiest people in the world; as of July 2024, Forbes estimates his net worth to be US$221 billion.[4]\n\nMusk was born in Pretoria to model Maye and businessman and engineer Errol Musk, and briefly attended the \nUniversity of Pretoria before immigrating to Canada at age 18, acquiring citizenship through his Canadian-born mother. Two years later, \nhe matriculated at Queen's University at Kingston in Canada. Musk later transferred to the University of Pennsylvania and received bachelor's \ndegrees in economics and ph

In [21]:
# Using LLM we will convert the data into graph document

In [23]:
# downloading graph transofrmer
!pip install --upgrade --quiet langchain_experimental

In [25]:
# loading the graph transformer
from langchain_experimental.graph_transformers import LLMGraphTransformer
llmtransformer = LLMGraphTransformer(
    llm=llm
)

In [47]:
# converting the document into node, relationship and edges
graphdocument = llmtransformer.convert_to_graph_documents(documents)

In [48]:
graphdocument

[GraphDocument(nodes=[Node(id='Elon Reeve Musk', type='Person'), Node(id='Spacex', type='Company'), Node(id='Tesla, Inc.', type='Company'), Node(id='X Corp.', type='Company'), Node(id='X', type='Company'), Node(id='The Boring Company', type='Company'), Node(id='Xai', type='Company'), Node(id='Neuralink', type='Company'), Node(id='Openai', type='Company'), Node(id='Forbes', type='Organization'), Node(id='University Of Pretoria', type='University'), Node(id='Canada', type='Country'), Node(id="Queen'S University At Kingston", type='University'), Node(id='University Of Pennsylvania', type='University'), Node(id='California', type='State'), Node(id='Stanford University', type='University'), Node(id='Zip2', type='Company'), Node(id='Maye Musk', type='Person'), Node(id='Errol Musk', type='Person'), Node(id='Kimbal Musk', type='Person')], relationships=[Relationship(source=Node(id='Elon Reeve Musk', type='Person'), target=Node(id='Spacex', type='Company'), type='HAS_KEY_ROLE'), Relationship(so

In [49]:
graphdocument[0].nodes

[Node(id='Elon Reeve Musk', type='Person'),
 Node(id='Spacex', type='Company'),
 Node(id='Tesla, Inc.', type='Company'),
 Node(id='X Corp.', type='Company'),
 Node(id='X', type='Company'),
 Node(id='The Boring Company', type='Company'),
 Node(id='Xai', type='Company'),
 Node(id='Neuralink', type='Company'),
 Node(id='Openai', type='Company'),
 Node(id='Forbes', type='Organization'),
 Node(id='University Of Pretoria', type='University'),
 Node(id='Canada', type='Country'),
 Node(id="Queen'S University At Kingston", type='University'),
 Node(id='University Of Pennsylvania', type='University'),
 Node(id='California', type='State'),
 Node(id='Stanford University', type='University'),
 Node(id='Zip2', type='Company'),
 Node(id='Maye Musk', type='Person'),
 Node(id='Errol Musk', type='Person'),
 Node(id='Kimbal Musk', type='Person')]

In [50]:
graphdocument[0].relationships

[Relationship(source=Node(id='Elon Reeve Musk', type='Person'), target=Node(id='Spacex', type='Company'), type='HAS_KEY_ROLE'),
 Relationship(source=Node(id='Elon Reeve Musk', type='Person'), target=Node(id='Tesla, Inc.', type='Company'), type='HAS_KEY_ROLE'),
 Relationship(source=Node(id='Elon Reeve Musk', type='Person'), target=Node(id='X Corp.', type='Company'), type='OWNS'),
 Relationship(source=Node(id='X Corp.', type='Company'), target=Node(id='X', type='Company'), type='OPERATES'),
 Relationship(source=Node(id='Elon Reeve Musk', type='Person'), target=Node(id='The Boring Company', type='Company'), type='FOUNDING_MEMBER'),
 Relationship(source=Node(id='Elon Reeve Musk', type='Person'), target=Node(id='Xai', type='Company'), type='FOUNDING_MEMBER'),
 Relationship(source=Node(id='Elon Reeve Musk', type='Person'), target=Node(id='Neuralink', type='Company'), type='FOUNDING_MEMBER'),
 Relationship(source=Node(id='Elon Reeve Musk', type='Person'), target=Node(id='Openai', type='Compan

In [52]:
# Load the converted data into Neo4j
graph.add_graph_documents(graphdocument)

In [53]:
### Load the dataset of movie
# Load data into neo4j and query the database using LLM

movie_query="""
LOAD CSV WITH HEADERS FROM
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row

MERGE(m:Movie{id:row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)
FOREACH (director in split(row.director, '|') |
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
FOREACH (actor in split(row.actors, '|') |
    MERGE (p:Person {name:trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))
FOREACH (genre in split(row.genres, '|') |
    MERGE (g:Genre {name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))
"""

In [54]:
graph

In [55]:
# run the above query
graph.query(movie_query)

[]

In [56]:
# we create chain and we can ask questions as such

from langchain.chains import GraphCypherQAChain
chain = GraphCypherQAChain.from_llm(
    llm, graph=graph, verbose=True
)

In [39]:
# query the db using invoke command
chain.invoke({"query":"who was the director of movie GoldenEye?"})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title: "GoldenEye"})<-[:DIRECTED]-(p:Person) RETURN p.name 

Full Context:
[{'p.name': 'Martin Campbell'}]

> Finished chain.


{'query': 'who was the director of movie GoldenEye?',
 'result': 'Martin Campbell was the director of movie GoldenEye. \n'}

In [40]:
chain.invoke({"query":"who was the actor of movie GoldenEye?"})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title: "GoldenEye"})<-[:ACTED_IN]-(p:Person) RETURN p.name 

Full Context:
[{'p.name': 'Pierce Brosnan'}, {'p.name': 'Famke Janssen'}, {'p.name': 'Sean Bean'}, {'p.name': 'Izabella Scorupco'}]

> Finished chain.


{'query': 'who was the actor of movie GoldenEye?',
 'result': 'Pierce Brosnan, Famke Janssen, Sean Bean, and Izabella Scorupco were the actors of the movie GoldenEye. \n'}